In [6]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


class CollabRecommendation:
    def __init__(self, user_data, product_data, ratings_data):
        self.user_data = user_data
        self.product_data = product_data
        self.ratings_data = ratings_data

    @staticmethod
    def calculate_similarity(user1, user2):
        gender_sim = 1 if user1['gender'] == user2['gender'] else 0
        age_sim = 1 - abs(user1['age'] - user2['age']) / 100
        if user1['city'] == user2['city']:
            location_sim = 1
        elif user1['country'] == user2['country']:
            location_sim = 0.5
        else:
            location_sim = 0
        total_similarity = 0.2 * gender_sim + 0.6 * age_sim + 0.2 * location_sim
        return total_similarity

    def find_top_similar_users(self, user_info):
        similarities = []
        for _, other_user in self.user_data.iterrows():
            similarity = self.calculate_similarity(user_info, other_user)
            similarities.append((other_user['userID'], similarity))
        similarities.sort(key=lambda x: x[1], reverse=True)
        return [user[0] for user in similarities[:50]]

    def collaborative_filtering_recommendations(self, user_id, user_info, top_n=10):
        top50_similar_users = self.find_top_similar_users(user_info)
        filtered_ratings_data = self.ratings_data[self.ratings_data['userid'].isin(top50_similar_users)]
        user_item_matrix = filtered_ratings_data.pivot_table(
            index='userid',
            columns='productid',
            values='rating',
            aggfunc='mean'
        ).fillna(0)

        user_similarity = cosine_similarity(user_item_matrix)
        recommended_items = set()
        for target_user_id in top50_similar_users:
            try:
                target_user_index = user_item_matrix.index.get_loc(target_user_id)
            except KeyError:
                continue

            user_similarities = user_similarity[target_user_index]
            similar_users_indices = user_similarities.argsort()[::-1][1:]

            for user_index in similar_users_indices:
                rated_by_similar_user = user_item_matrix.iloc[user_index]
                not_rated_by_target_user = (rated_by_similar_user > 0) & (user_item_matrix.iloc[target_user_index] == 0)
                recommended_items.update(user_item_matrix.columns[not_rated_by_target_user][:top_n])
                if len(recommended_items) >= top_n:
                    break
        recommended_items_details = self.ratings_data[
            self.ratings_data['productid'].isin(recommended_items)
        ][['productid', 'rating']].drop_duplicates()
        recommended_items_with_names = recommended_items_details.merge(
            self.product_data[['id', 'name']],
            left_on='productid',
            right_on='id',
            how='inner'
        )
        top_recommendations = recommended_items_with_names.sort_values(
            by='rating', ascending=False
        ).head(top_n)

        return top_recommendations[['productid', 'name']].values.tolist()

    def recommend(self, user_id, user_info, top_n=20):
        return self.collaborative_filtering_recommendations(user_id, user_info, top_n=top_n)


# Load datasets
user_data = pd.read_csv('user_data.csv')
product_data = pd.read_csv('test_data.csv')
ratings_data = pd.read_csv('ratings_test.csv')

# Initialize the model
model = CollabRecommendation(user_data, product_data, ratings_data)

# User information
user_id = 41205
user_info = {
    'age': 22,
    'gender': 'female',
    'city': 'thomasville',
    'country': 'us',
    'interest': '18,19'
}

# Get recommendations
recommendations = model.recommend(user_id, user_info)

# Print recommendations
print(f"Recommendations for user ID {user_id}:")
for product_id, product_name in recommendations:
    print(f"Product ID: {product_id}, Product Name: {product_name}")


Recommendations for user ID 41205:
Product ID: acs131531, Product Name: Pinkcity Gems 4.45 Carat 5.25 Ratti Unheated Untreated Ceylon Yellow Sapphire Pukhraj Stone Certified Natural Gemstone AA...
Product ID: hkt081335, Product Name: Vaccman 0.2cc, 0.25cc, 0.3cc, 0.4cc 0.5cc, 0.6cc & 0.75cc Vaccinator : Automatic Vaccination Equipment : Poultry Vaccine I...
Product ID: acs216603, Product Name: GUESS Analog Rose Gold Dial Unisex Adult Watch-GW0314L3
Product ID: acs132437, Product Name: Costfide Men Messenger Bags Brown
Product ID: hkt111998, Product Name: SmartSlide Stainess Steel Modular Kitchen Basket Plate Fixer/Tandem Basket Dish Rack/Kitchen Rack/Kitchen Stand (Dish Rack...
Product ID: acs238258, Product Name: Pay Per E-Com Women's Saree Blouse Back Accessories Jewellery for Girl and Women Brooch Pin with Kundan and Pearl (White)
Product ID: app164090, Product Name: Kenvi US Priya Disco Room Smart Heater || 1000 Watt || Single Rod Heater || Room Heater || Winter Heater || Color-Mul